In [9]:
#Right now we could maybe widdle down to movies above a certain rating + number of ratings

In [1]:
import pandas as pd
import csv
import time
import numpy as np
import itertools

In [2]:
df_ratings = pd.read_csv('data/ratings.tsv',sep = '\t')
titles = pd.read_csv('data/titles.tsv',sep='\t')
names = pd.read_csv('data/names.tsv',sep = '\t')
movies = titles[titles['titleType']=='movie']
movies = list(movies['tconst'].unique())

C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
names.head()
titles.head()

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
0,tt0000001,short,Carmencita,Carmencita,0,1894,\N,1,"Documentary,Short"
1,tt0000002,short,Le clown et ses chiens,Le clown et ses chiens,0,1892,\N,5,"Animation,Short"
2,tt0000003,short,Pauvre Pierrot,Pauvre Pierrot,0,1892,\N,4,"Animation,Comedy,Romance"
3,tt0000004,short,Un bon bock,Un bon bock,0,1892,\N,12,"Animation,Short"
4,tt0000005,short,Blacksmith Scene,Blacksmith Scene,0,1893,\N,1,"Comedy,Short"


In [4]:
##get a list of movies based on a rating threshold and year threshold
def filterMovies(ratingThreshold,yearThreshold,dfratings,dftitles):
    avgVotes = np.mean(df_ratings['numVotes'])
    qualifiedMovies = []
    len(df_ratings)
    boolean_series = dfratings.tconst.isin(movies)
    dfratings = dfratings[boolean_series]
    for row in dfratings.iterrows():
        if row[1]['averageRating'] >= ratingThreshold and row[1]['numVotes'] >= avgVotes:
            qualifiedMovies.append(row[1]['tconst'])
    qualifiedMoviesFinal = []
    for movie in qualifiedMovies:
        year = (list(dftitles[dftitles['tconst']==movie]['startYear'])[0])
        if type(year) == str:
            year = int(year)
        if year >= yearThreshold:
            qualifiedMoviesFinal.append(movie)
    return qualifiedMoviesFinal

In [5]:
print(len(qualifiedMovies))

NameError: name 'qualifiedMovies' is not defined

In [6]:
qualifiedMovies = filterMovies(7.5,2000,df_ratings,titles)
print(len(qualifiedMovies))

2468


In [7]:
#find the frequent actors in the list of movies that meets the support threshold
def findFreqActors(qMovies,supportThreshold):
    count = 0
    freqActors = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            if name not in qMovies:
                continue
            else:
                actors = group['nconst'].unique()
                for actor in actors:
                    if actor in freqActors:
                        freqActors[actor][0] += 1
                        freqActors[actor][1].append(name)
                    else:
                        freqActors[actor] = [1,[name]]
    freqActorsFinal = dict()
    for actor in freqActors:
        if freqActors[actor][0] >= supportThreshold:
            freqActorsFinal[actor] = freqActors[actor]
    return freqActors

In [10]:
freqActors = findFreqActors(qualifiedMovies,2)

In [11]:
print(len(freqActors))

3423


In [12]:
def findPairs():
    freq_pairs = dict()
    for chunk in pd.read_csv('data/principals.tsv' ,sep='\t', chunksize = 1000000, usecols = ['tconst','category','nconst']):
        chunk = chunk[chunk['category']=='actor']
        chunk = chunk.groupby('tconst')
        for name, group in chunk:
            freqActorsGroup = [actor for actor in group['nconst'] if actor in freqActors]
            freqPairsGroup = list(itertools.combinations(freqActorsGroup,2))
            for pair in freqPairsGroup:
                if pair in freq_pairs:
                    freq_pairs[pair][0] += 1
                    freq_pairs[pair][1].append(name)
                else:
                    freq_pairs[pair] = [1,[name]]
        freq_pairs = dict(sorted(freq_pairs.items(), key=lambda item: item[1],reverse=True))
        freq_pairsFinal = dict()
        for pair in freq_pairs:
            if (pair[1],pair[0]) not in freq_pairsFinal:
                freq_pairsFinal[pair] = freq_pairs[pair]
        return freq_pairsFinal

In [13]:
freq_pairs = findPairs()

In [18]:
print(len(freq_pairs))
freqpairsfinal = dict()
for pair in freq_pairs:
    if (freq_pairs[pair][0] >= 3):
        freqpairsfinal[pair] = freq_pairs[pair]
    #print(getName(pair[0]),getName(pair[1]))

1203


In [15]:
def getName(nconst):
    return list(names[names['nconst']==nconst]['primaryName'])[0] 
def getSoloMovies(name1,name2,pairsDict,soloDict):
    name1solomovies = []
    name2solomovies = []
    for movie in soloDict[name1][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name1solomovies.append(movie)
    for movie in soloDict[name2][1]:
        if movie not in pairsDict[(name1,name2)][1]:
            name2solomovies.append(movie)
    return name1solomovies,name2solomovies
def getRatings(movies):
    ratings = []
    if len(movies) == 0:
        return 0
    for movie in movies:
        ratings.append(list(df_ratings[df_ratings['tconst']== movie]['averageRating'])[0])
    rating = np.mean(ratings)
    return rating

In [25]:
for pair in freqpairsfinal:
    movies1, movies2 = getSoloMovies(pair[0],pair[1],freqpairsfinal,freqActors)
    ratings1 = getRatings(movies1)
    ratings2 = getRatings(movies2)
    ratings_together = getRatings(freqpairsfinal[pair][1])
    betteralone1 = True
    betteralone2 = True
    bt1 = False
    bt2 = False
    bettertogether = False
    if ratings_together >= ratings1:
        betteralone1 = False
        bt1 = True
    if ratings_together >= ratings2:
        betteralone2 = False
        bt2 = True
    if bt1 and bt2:
        bettertogether = True
    if bettertogether:
        print(movies1)
        print(movies2)
        print(freqpairsfinal[pair][1])
        print("For the duo " + getName(pair[0]) + " , " + getName(pair[1]) + "Their rating of " + str(ratings_together) + "was betteer than both of their solo ratings: " + str(ratings1) + ", " + str(ratings2))


['tt3872778']
['tt3872778']
['tt0063929', 'tt0066765', 'tt0071853', 'tt0079470', 'tt0085959']
For the duo John Cleese , Terry GilliamTheir rating of 8.04was betteer than both of their solo ratings: 7.7, 7.7
['tt3872778']
['tt3872778']
['tt0063929', 'tt0071853', 'tt0084352', 'tt0085959']
For the duo John Cleese , Eric IdleTheir rating of 8.1was betteer than both of their solo ratings: 7.7, 7.7
['tt1045658', 'tt1302006', 'tt7286456']
['tt1302006']
['tt0081398', 'tt0099685', 'tt0112641']
For the duo Robert De Niro , Joe PesciTheir rating of 8.366666666666665was betteer than both of their solo ratings: 7.966666666666666, 7.8


In [30]:
titles[titles['tconst']=='tt1302006']

,tconst,titleType,primaryTitle,originalTitle,isAdult,startYear,endYear,runtimeMinutes,genres
2692725,tt1302006,movie,The Irishman,The Irishman,0,2019,\N,209,"Biography,Crime,Drama"


C:\Users\Matt\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
